In [1]:
!pip install -q tfds-nightly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 20.9 MB/s eta 0:00:00


In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
import os
%load_ext tensorboard
print(tf.__version__)

2.18.0


In [3]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.TJ8VZB_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.TJ8VZB_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


Number of devices: 1


In [5]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))


Number of devices: 1


In [6]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_dataset = mnist_train.map(scale).cache().shuffle(10000).batch(64 * strategy.num_replicas_in_sync)
eval_dataset = mnist_test.map(scale).batch(64 * strategy.num_replicas_in_sync)


In [7]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath="./checkpoints/ckpt_{epoch}.weights.h5", save_weights_only=True)
]


In [10]:
EPOCHS = 12
model.fit(train_dataset, epochs=EPOCHS, callbacks=callbacks)


Epoch 1/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 36s 36ms/step - accuracy: 0.8851 - loss: 0.4208
Epoch 2/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 30s 32ms/step - accuracy: 0.9782 - loss: 0.0756
Epoch 3/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.9851 - loss: 0.0510
Epoch 4/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.9886 - loss: 0.0374
Epoch 5/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 30s 32ms/step - accuracy: 0.9915 - loss: 0.0298
Epoch 6/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - accuracy: 0.9934 - loss: 0.0227
Epoch 7/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 30ms/step - accuracy: 0.9941 - loss: 0.0184
Epoch 8/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 30ms/step - accuracy: 0.9952 - loss: 0.0153
Epoch 9/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.9965 - loss: 0.0110
Epoch 10/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 30ms/step - accuracy: 0.9974 - loss: 0.0081
Epoch 11/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 31ms/step - accuracy: 0.9977 - loss: 0.0072
Epoch 12/12
938/938 ━━━━━━━━━━

In [13]:
import os

checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)


In [14]:
EPOCHS = 12
model.fit(train_dataset, epochs=EPOCHS, callbacks=callbacks)


Epoch 1/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 32s 34ms/step - accuracy: 0.9985 - loss: 0.0050
Epoch 2/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 30ms/step - accuracy: 0.9986 - loss: 0.0049
Epoch 3/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 30ms/step - accuracy: 0.9987 - loss: 0.0040
Epoch 4/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.9990 - loss: 0.0036
Epoch 5/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 30ms/step - accuracy: 0.9997 - loss: 0.0014
Epoch 6/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 30ms/step - accuracy: 0.9983 - loss: 0.0061
Epoch 7/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 30ms/step - accuracy: 0.9994 - loss: 0.0020
Epoch 8/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.9984 - loss: 0.0052
Epoch 9/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 30ms/step - accuracy: 0.9993 - loss: 0.0020
Epoch 10/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 29ms/step - accuracy: 0.9989 - loss: 0.0035
Epoch 11/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 29ms/step - accuracy: 0.9994 - loss: 0.0019
Epoch 12/12
938/938 ━━━━━━━━━━

In [15]:
ls ./checkpoints

ckpt_10.weights.h5  ckpt_1.weights.h5  ckpt_4.weights.h5  ckpt_7.weights.h5
ckpt_11.weights.h5  ckpt_2.weights.h5  ckpt_5.weights.h5  ckpt_8.weights.h5
ckpt_12.weights.h5  ckpt_3.weights.h5  ckpt_6.weights.h5  ckpt_9.weights.h5


In [17]:
# Load the latest checkpoint file
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)

if latest_checkpoint:
    model.load_weights(latest_checkpoint)
    eval_loss, eval_acc = model.evaluate(eval_dataset)
    print('Eval loss: {}, Eval accuracy: {}'.format(eval_loss, eval_acc))
else:
    print("No checkpoint found.")


No checkpoint found.


In [18]:
!ls ./checkpoints


ckpt_10.weights.h5  ckpt_1.weights.h5  ckpt_4.weights.h5  ckpt_7.weights.h5
ckpt_11.weights.h5  ckpt_2.weights.h5  ckpt_5.weights.h5  ckpt_8.weights.h5
ckpt_12.weights.h5  ckpt_3.weights.h5  ckpt_6.weights.h5  ckpt_9.weights.h5


In [19]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath="./checkpoints/ckpt_{epoch}.weights.h5",
                                       save_weights_only=True,
                                       save_best_only=False)
]



In [22]:
import os
checkpoint_dir = './checkpoints'  # Replace with your checkpoint directory
print("Checkpoint files:", os.listdir(checkpoint_dir))


Checkpoint files: ['ckpt_4.weights.h5', 'ckpt_5.weights.h5', 'ckpt_6.weights.h5', 'ckpt_9.weights.h5', 'ckpt_12.weights.h5', 'ckpt_8.weights.h5', 'ckpt_11.weights.h5', 'ckpt_3.weights.h5', 'ckpt_1.weights.h5', 'ckpt_10.weights.h5', 'ckpt_7.weights.h5', 'ckpt_2.weights.h5']


In [23]:
# Try loading a specific checkpoint manually
model.load_weights('./checkpoints/ckpt_10.weights.h5')


In [24]:
model.save_weights('./checkpoints/ckpt_1.weights.h5')


In [25]:
eval_loss, eval_acc = model.evaluate(eval_dataset)
print(f"Eval Loss: {eval_loss}, Eval Accuracy: {eval_acc}")


157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9874 - loss: 0.0577
Eval Loss: 0.06823325902223587, Eval Accuracy: 0.9861000180244446
